### 1. Explicação do Objetivo e Motivação:
Explique o objetivo do dashboard que você está desenvolvendo e a motivação por trás da escolha dos dados e funcionalidades que serão implementados.

Base: Chegada mensal de turistas pelo Rio de Janeiro, por via Aérea, segundo continentes e países de residência permanente, entre 2006-2019
Disponível em: https://www.data.rio/documents/a6c6c3ff7d1947a99648494e0745046d/about

- Objetivo: Apresentar a dados de turismo no Rio de Janeiro.
- Motivador: Conseguir identificar o crescimento ou redução da quantidade de turistas de acordo com o país de origem.
- Funcionalidades: 
    - Filtros por período, continente e país
    - Gráfico de linha exibindo tendências
    - Gráfico de pizza com número de turistas por continente
    - Gráfico de pareto com número absoluto de turistas por país
    - Histograma por mês
    - Mapa de calor por mês e país
    - Mapa de rotas

### 2. Realizar Upload de Arquivo CSV:
Crie uma interface em Streamlit que permita ao usuário fazer o upload de um arquivo CSV contendo dados de turismo do portal Data.Rio.

### 3. Filtro de Dados e Seleção:
Implemente seletores (radio, checkbox, dropdowns) na interface que permitam ao usuário filtrar os dados carregados e selecionar as colunas ou linhas que deseja visualizar.

### 4. esenvolver Serviço de Download de Arquivos:
Implemente um serviço que permita ao usuário fazer o download dos dados filtrados em formato CSV diretamente pela interface da aplicação.

### 5. Utilizar Barra de Progresso e Spinners:
Adicione uma barra de progresso e um spinner para indicar o carregamento dos dados enquanto o arquivo CSV é processado e exibido na interface.

### 6. Utilizar Color Picker:
Adicione um color picker à interface que permita ao usuário personalizar a cor de fundo do painel e das fontes exibidas na aplicação.

### 7. Utilizar Funcionalidade de Cache:
Utilize a funcionalidade de cache do Streamlit para armazenar os dados carregados de grandes arquivos CSV, evitando a necessidade de recarregá-los a cada nova interação.

### 8. Persistir Dados Usando Session State:
Implemente a persistência de dados na aplicação utilizando Session State para manter as preferências do usuário (como filtros e seleções) durante a navegação.

### 9. Criar Visualizações de Dados - Tabelas:
Crie uma tabela interativa que exiba os dados carregados e permita ao usuário ordenar e filtrar as colunas diretamente pela interface.

### 10. Criar Visualizações de Dados - Gráficos Simples:
Desenvolva gráficos simples (como barras, linhas, e pie charts) para visualização dos dados carregados, utilizando o Streamlit.

### 11. Criar Visualizações de Dados - Gráficos Avançados:
Adicione gráficos mais avançados (como histograma ou scatter plot) para fornecer insights mais profundos sobre os dados.

### 12. Exibir Métricas Básicas:
Implemente a exibição de métricas básicas (como contagem de registros, médias, somas) diretamente na interface para fornecer um resumo rápido dos dados carregados.